In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import pywt

In [2]:
X = []
Y = []

## Load data

In [15]:
IDD_rest = np.load("Data/GeneratedData/combined_data/IDD_rest.npy")
IDD_music = np.load("Data/GeneratedData/combined_data/IDD_music.npy")

TDC_rest = np.load("Data/GeneratedData/combined_data/TDC_rest.npy")
TDC_music = np.load("Data/GeneratedData/combined_data/TDC_music.npy")

1610

In [ ]:
X = IDD_rest
Y = []



In [4]:
X = np.asarray(X)
Y = np.asarray(Y)


In [5]:
print(X.shape)
print(Y.shape)

(3220, 14, 640)
(3220,)


In [6]:
# reshape data
X = X.reshape(X.shape[0], 14, 640, 1)
X.shape

(3220, 14, 640, 1)

## Test-Train split

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=22)

In [8]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=22)

In [9]:
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_val shape:", X_val.shape)
print("y_val shape:", y_val.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)


X_train shape: (1932, 14, 640, 1)
y_train shape: (1932,)
X_val shape: (644, 14, 640, 1)
y_val shape: (644,)
X_test shape: (644, 14, 640, 1)
y_test shape: (644,)


In [11]:
import pywt

def wavelet_coefs(data,func, wavelet_name='db4'):
    coefs = pywt.wavedec(data, wavelet=wavelet_name, level=None)
    n_channels, n_times = data.shape
    levdec = len(coefs) - 1

    wavelet_energy = np.zeros((n_channels, levdec))
    for j in range(n_channels):
        for l in range(levdec):
            if func=='energy':
                wavelet_energy[j, l] = np.sum(coefs[levdec - l][j, :] ** 2)
            else:
                wavelet_energy[j, l] = func(coefs[levdec - l][j, :])
    return wavelet_energy.ravel()

In [12]:
def calculate_features(func):
    features1=[]
    features2=[]
    for k in tqdm(range(7)):
        feature1=[]
        feature2=[]
        for i in range(30):  
            feature1.append(wavelet_coefs(IDD_rest[k][i],func=func))
            feature2.append(wavelet_coefs(TDC_rest[k][i],func=func))
        features1.append(np.array(feature1))
        features2.append(np.array(feature2))
    x1=np.array(features1)        
    x2=np.array(features2) 
    y=np.concatenate(((np.zeros(x1.shape[0])),(np.ones(x2.shape[0]))))
    X=np.concatenate((x1,x2),axis=0)
    return X,y

In [ ]:
X, y = calculate_features(np.mean)